# VCintModel

Machine information 

In [1]:
versioninfo()

Julia Version 1.2.0
Commit c6da87ff4b (2019-08-20 00:03 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-6267U CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


## Data

Suppose we have the following variables:

* `y`: $324$-by-$1$ vector of phenotype of interest 
* `X`: $324$-by-$1$ vector indicating sex of each individual 
* `G`: $150$-by-$1$ vector whose $i$-th element is a $324$-by-$q_i$ genotype matrix (matrix of minor allele counts) for gene $i,$ where $i=1,\ldots, 150$ and $q_i$ is the number of variants for gene $i$
* `trtvec`: $324$-by-$1$ vector indicating treatment status of each individual. 

These variables are saved in `.jld2` file. First, read in variables using `JLD2` package. 

In [4]:
using JLD2
@load "/Users/juhyun-kim/Box Sync/workspace/vcselect/codebase/julia/VarianceComponentSelect.jl/docs/SNPset_interaction.jld2" y y2 X G V trtvec

6-element Array{Symbol,1}:
 :y     
 :y2    
 :X     
 :G     
 :V     
 :trtvec

Our phenotype vector: 

In [5]:
y

324-element Array{Float64,1}:
 -0.40911273847444835 
 -0.0597995644528963  
  0.02798994878695149 
 -0.5810287222548617  
  0.4325440331155418  
  0.048256153207898195
 -0.27027696763685594 
  0.10490944338119751 
 -0.7799009563553139  
 -0.20593444247247622 
  0.5520065403740653  
 -0.10092686337307819 
  0.5983104367265357  
  ⋮                   
  0.03739468023738035 
  0.47352506877371714 
 -0.22836115042409924 
 -0.3876214190551975  
  0.10326243849550518 
 -0.4882830145406528  
 -0.06903684417417116 
  0.09798041795966334 
 -0.1480073641765541  
 -0.29900800903355274 
 -0.008878790322644575
 -0.09571535379121696 

Covariate matrix indicating sex:

In [6]:
X

324-element Array{Float64,1}:
 1.0
 2.0
 2.0
 1.0
 2.0
 1.0
 2.0
 2.0
 2.0
 1.0
 2.0
 1.0
 1.0
 ⋮  
 1.0
 1.0
 2.0
 2.0
 2.0
 1.0
 1.0
 2.0
 1.0
 1.0
 1.0
 2.0

Vector of genotype matrix for each gene: 

In [7]:
G

150-element Array{Array{Float64,2},1}:
 [2.0 0.0 … 2.0 2.0; 2.0 1.0 … 1.4485981308411215 2.0; … ; 2.0 0.0 … 1.0 2.0; 2.0 0.0 … 2.0 2.0]
 [2.0 2.0 … 2.0 2.0; 2.0 1.0 … 1.0 0.0; … ; 2.0 1.0 … 1.0 0.0; 2.0 2.0 … 1.0 2.0]               
 [2.0 2.0 … 0.0 2.0; 2.0 2.0 … 2.0 2.0; … ; 2.0 2.0 … 1.0 2.0; 2.0 2.0 … 0.0 1.0]               
 [2.0 1.0 … 0.0 2.0; 2.0 1.0 … 0.0 2.0; … ; 2.0 0.0 … 1.0 2.0; 1.0 1.0 … 1.0 2.0]               
 [1.0 2.0 … 0.0 2.0; 1.0 2.0 … 1.0 2.0; … ; 0.0 2.0 … 1.0 2.0; 2.0 2.0 … 0.0 2.0]               
 [2.0 2.0 … 1.0 2.0; 2.0 2.0 … 1.0 2.0; … ; 2.0 2.0 … 1.0 2.0; 2.0 2.0 … 1.0 2.0]               
 [2.0 1.0 … 1.0 2.0; 2.0 1.0 … 1.0 2.0; … ; 2.0 2.0 … 0.0 2.0; 2.0 2.0 … 1.0 2.0]               
 [1.0 1.0 … 1.0 1.0; 1.0 2.0 … 1.0 0.0; … ; 2.0 2.0 … 2.0 1.0; 2.0 2.0 … 2.0 1.0]               
 [2.0 2.0 … 2.0 0.0; 2.0 1.0 … 1.0 0.0; … ; 1.0 2.0 … 2.0 0.0; 2.0 2.0 … 2.0 1.0]               
 [2.0 2.0 … 2.0 1.0; 2.0 2.0 … 2.0 1.0; … ; 2.0 2.0 … 2.0 1.0; 2.0 0.0 … 2.0 1.0]       

Elements in `G` are matrices of size $324\times q_i.$ Number of rows must be the same (n=324) because each row is for each individual. On the other hand, number of columns vary because number of SNPs/variants vary from gene to gene. Here we list different sizes elements of `G` have: 

In [10]:
unique(size.(G))

8-element Array{Tuple{Int64,Int64},1}:
 (324, 44)
 (324, 27)
 (324, 47)
 (324, 14)
 (324, 42)
 (324, 29)
 (324, 30)
 (324, 18)

Vector of treatment status:

In [8]:
trtvec

324-element Array{Int64,1}:
 0
 0
 0
 0
 1
 1
 0
 1
 0
 1
 1
 1
 1
 ⋮
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 0
 1

Let us create a diagonal matrix whose elements are from `trtvec`:


In [13]:
using LinearAlgebra
T = Diagonal(trtvec)

324×324 Diagonal{Int64,Array{Int64,1}}:
 0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  0  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  0  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅ 

Now based on what we have, we will create two vectors of covariance matrices:

* `V`: 151-by-1 vector of 324-by-324 matrices 
    - `V[i]` $ = G_i G_i^T / ||G_i G_i^T||_F$ where $i=1,\ldots,150$
    - `V[end]` $= I_{324} / \sqrt{324}$
* `Vint`: 150-by-1 vector of 324-by-324 matrices
    - `Vint[i]` $ = T G_i G_i^T T^T / ||T G_i G_i^T T^T||_F$ where $i = 1,\ldots, 150.$
    
Note that we divide by Frobenius norm ($||\cdot||_F$) to put matrices on the same scale.

In [15]:
n, m = length(y), length(G)
V = Vector{Matrix{Float64}}(undef, m + 1)
Vint = Vector{Matrix{Float64}}(undef, m)
    
for i in 1:m
    V[i] = G[i] * G[i]'
    Vint[i] = T * V[i] * T' 
    V[i] ./= norm(V[i])
    Vint[i] ./= norm(Vint[i])
end 
V[end] = Matrix(I, n, n) ./ √n;

In [19]:
V

151-element Array{Array{Float64,2},1}:
 [0.003291667254666042 0.003021726705196923 … 0.0030853699763006446 0.002995120655146579; 0.003021726705196923 0.0031759677171174796 … 0.0029207450416141385 0.0029624173852930303; … ; 0.0030853699763006446 0.0029207450416141385 … 0.0033536029685539274 0.0030544299750504714; 0.002995120655146579 0.0029624173852930303 … 0.0030544299750504714 0.0034695952143777202]    
 [0.0038736704792971055 0.0031542745331419285 … 0.0034586343565152725 0.00367998695533225; 0.0031542745331419285 0.0030989363834376844 … 0.0032372817576982954 0.0030989363834376844; … ; 0.0034586343565152725 0.0032372817576982954 … 0.0037353251050364945 0.0034032962068110285; 0.00367998695533225 0.0030989363834376844 … 0.0034032962068110285 0.003790663254740739]    
 [0.003134943134737379 0.0029621509934526415 … 0.003011520176676852 0.0027399896689436934; 0.0029621509934526415 0.003283050684410011 … 0.0030608893599010627 0.0028387280353921146; … ; 0.003011520176676852 0.003060889359901

In [20]:
Vint

150-element Array{Array{Float64,2},1}:
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007022323013197897] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.00773729902969992]  
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0064020557433550306]
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007187137793403356] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007319758091161073] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007135751936863028] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.006472228768822936] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007290657333798592] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.007352558297122053] 
 [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0

## Model formulation 

We assume the response vector $y$ is distributed as follows:

$$y \sim N(X\beta, \sum_{i=1}^{m} \left( \sigma_{i1}^2 V_{i1} +  \sigma_{i2}^2 V_{i2}\right) + \sigma_0^2 I_n ),$$

such that each gene/group (indexed by $i$) is associated with two variance components
- $\sigma_{i1}^2$: for the gene/group itself 
- $\sigma_{i2}^2$: for the interaction between gene/group and treatment status indicated by $T$. 

Suppose we want to include/exclude main effects and interaction term together as a pair and identify the pairs that are associated with the response $y.$ In other words, all-in or all-out! This can be achieved by setting up `VCintModel` in `VarianceComponentSelect` package.

## VCintModel

First load the package. 

In [23]:
using VarianceComponentSelect

To perform selection, take 2 steps:

**Step 1 (Construct a model)**. Construct an instance of `VCintModel`, which is the fundamental type for variance component interaction model. It consists of fields 

* `Y`: $n$-by-$1$ responses. 
* `X`: $n$-by-$p$ covariate matrix (if exists).
* `V=(V[1],...,V[m],I)`: a vector of $n$-by-$n$ covariance matrices. The last covariance matrix must be positive definite and usually is the identity matrix.
* `Vint=(Vint[1],...,Vint[m])`: a vector of $n$-by-$n$ covariance matrices. 
* `σ2=(σ2[1],...,σ2[m],σ2[0])`: a vector of initial estimates for variance component parameters. If not supplied, it is set to be a vector of ones by default. 
* `σ2int=(σ2int[1],...,σ2int[m])`: a vector of initial estimates for variance component parameters. If not supplied, it is set to be a vector of ones by default. 

`VCintModel` can be initialized by 

Let us construct a `VCintModel` using `Y`, `X`, `Vint` and `V`. Since we do not provide `σ2`, it is initialized to be a vector of ones. 

In [24]:
vcm = VCintModel(y, X, V, Vint);

In [24]:
vcm_nopen = deepcopy(vcm)
vcselect!(vcm_nopen)

(VCintModel{Float64}([-0.40911273847444835, -0.0597995644528963, 0.02798994878695149, -0.5810287222548617, 0.4325440331155418, 0.048256153207898195, -0.27027696763685594, 0.10490944338119751, -0.7799009563553139, -0.20593444247247622  …  -0.22836115042409924, -0.3876214190551975, 0.10326243849550518, -0.4882830145406528, -0.06903684417417116, 0.09798041795966334, -0.1480073641765541, -0.29900800903355274, -0.008878790322644575, -0.09571535379121696], [1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0  …  2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0], Array{Float64,2}[[0.003291667254666042 0.003021726705196923 … 0.0030853699763006446 0.002995120655146579; 0.003021726705196923 0.0031759677171174796 … 0.0029207450416141385 0.0029624173852930303; … ; 0.0030853699763006446 0.0029207450416141385 … 0.0033536029685539274 0.0030544299750504714; 0.002995120655146579 0.0029624173852930303 … 0.0030544299750504714 0.0034695952143777202], [0.0038736704792971055 0.0031542745331419285 … 0.003458634

In [25]:
vcm_nopen.Σ

151-element Array{Float64,1}:
  9.344776908017007e-85  
  8.09443812272092e-12   
  2.523361869734106e-146 
  6.207320835323817e-18  
  3.156089550043402e-49  
 10.568125138019838      
  1.2221938440362864e-39 
  6.036583813540689e-40  
  3.377473784710914e-43  
  6.278347228439025e-12  
  5.09976202397559e-114  
  9.534436189900314e-94  
  1.3687527049565546e-101
  ⋮                      
  1.2645638618857376e-95 
  1.3436915478054653e-31 
  4.31716228635469e-60   
  2.333260983756985e-29  
  1.7747453247761207e-28 
  4.781514204938161e-5   
  2.2570380322522497e-82 
  4.942985181533002e-9   
  4.050055103448707      
  3.331587787133644e-95  
 19.4692557133325        
  5.564148840746572e-8   

In [26]:
vcm_nopen.Σint

150-element Array{Float64,1}:
 9.60409006984543e-24   
 2.348765986875331e-9   
 3.270844627775662e-46  
 1.0920322616972776e-106
 2.1930515260316883e-23 
 1.6086805375489337e-79 
 1.610021228214981e-33  
 1.0961939699018768e-80 
 9.026946878154784e-20  
 2.2702488384024474e-51 
 4.04830370021412e-87   
 1.1832066359912632e-92 
 3.217712835429179e-154 
 ⋮                      
 7.322619784172027e-14  
 2.1687160114586293e-59 
 4.933466606901258e-53  
 9.714119951909439e-38  
 5.605942956520463e-33  
 3.481447131349968e-50  
 2.264224199063822e-117 
 6.375758067405585e-58  
 1.0156959344357028e-30 
 1.5934504314743633e-38 
 2.394505750281459e-63  
 3.730739732996209      